In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": "is it to late to join the course?"}]
)

In [10]:
response.choices[0].message.content

"It depends on the course and the institution offering it. Some courses may have specific enrollment deadlines, while others may allow students to join late with permission from the instructor or department. It's best to contact the course coordinator or admissions office to inquire about late enrollment options."

In [13]:
import minsearch
import json

In [14]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [15]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [16]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [18]:
q = 'the course has already started, can I still enroll?'

In [19]:
index.fit(documents)

In [20]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It depends on the policies of the institution offering the course. Some institutions may allow late enrollment with permission from the instructor, while others may have strict enrollment deadlines. It's best to contact the institution or instructor directly to inquire about late enrollment options."

In [22]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [23]:
query = 'how do I run kafka?'

rag(query)

'To run Kafka, you can follow these steps:\nFor Java Kafka - In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\nFor Python Kafka - Create a virtual environment and run requirements.txt and the python files in that environment. Use the provided installation command to install the \'dlt[duckdb]\' package if necessary. Additionally, to fix permission denied error, run chmod +x build.sh in the appropriate directory. If you encounter the "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'" error, consider using pip install kafka-python-ng instead.'